In [ ]:
# 获取当前notebook的绝对路径
import sys, os
from pathlib import Path
from pprint import pprint
current_path = Path.cwd()  # 或者使用 %pwd magic命令
project_root = current_path.parents[2]  # 如果notebook在project_root/notebooks/下
print("notebook所在的路径：", current_path)
print("project_root：", project_root)
sys.path.append(str(project_root))
import FastAPI_setup

#### 验证agent初始化无错误

In [2]:
from app.services.bp_msg import AgentMessage, AgentMessageHistory
from app.services.bp_state import AgentState, AgentStateHistory
from datetime import datetime

In [3]:
# project_id = "20563b61-7818-474b-9641-c40757bbd3fe"  # test1
project_id = "cc6bca8a-df5d-43d5-a931-875113df1f50"  # test2
# Project_id = "f6db0cbe-e7af-4300-8335-01ba4ffdbb93"  # small test

In [4]:
# 实例化agent cache和storage
from app.services.cache import Cache
cache = Cache(project_id)
from app.services.storage import Storage
storage = Storage(project_id)

In [ ]:
await cache.clean_up()
# 注意clean_up会调用storage的clear_storage方法，如果django无表，该方法会促发建表。

#### 1. 测试 Agent State History的存储

In [ ]:
# 构建mock数据
from app.services.bp_state import AgentState, StageEnum, StageStatus, AgentStateHistory
from datetime import datetime
# mock AgentState数据
agent_state_1 = AgentState(
    agent_id="123",
    overall_progress=10,
    active_stage=StageEnum.STRUCTURING,
    stage_status=StageStatus.IN_PROGRESS,
    stage_task_id="123",
    created_at=datetime.now(),
    updated_at=datetime.now()
)

agent_state_2 = AgentState(
    agent_id="123",
    overall_progress=20,
    active_stage=StageEnum.PLANNING,
    stage_status=StageStatus.IN_PROGRESS,
    stage_task_id="123",
    created_at=datetime.now(),
    updated_at=datetime.now()
)



In [ ]:
# 存储状态
await cache.save_agent_state(agent_state_1)

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_state, agent_state_history =  await cache.get_agent_state()
print(agent_state)
print(agent_state_history)

In [ ]:
# 删除缓存
key = cache.get_cache_keys().get('agent_state_history')
from app.core.redis_helper import RedisClient
await RedisClient.delete(key)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
# 检查django storage 数据是否还在？  理论上应该在
data = await storage.get_from_django(params={'fields': 'agent_state_history'})
print(data)
print(type(data))

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_state, agent_state_history =  await cache.get_agent_state()
print(agent_state)
print(agent_state_history)

In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
# 使用 cache的clean up来删除
await cache.clean_up(['agent_state_history'])

In [ ]:
# 理论上，缓存和storage 数据都不在了
data1 = await RedisClient.get(key)
print(data1)
data2 = await storage.get_from_django(params={'fields': 'agent_state_history'})
print(data2)
print(type(data2))

# 需要特别注意，以下返回
# 1）storage返回的不是空，而是一个dict，但content为空
# 2）由于content为空，dict不会进一步给到cache， 所以cache为空，而不是content为空的dict.  

In [ ]:
# 但无论怎么样，从最终Agent的视角，agent_state和agent_state_history都是空的。 
agent_state, agent_state_history =  await cache.get_agent_state()
print(agent_state)
print(agent_state_history)

#### 2. 测试文档的存储

In [16]:
# mock data
raw_document = {
    'type': 'doc',
    'content': [
        {
            'type': 'paragraph',
            'content': [
                {
                    'type': 'text',
                    'text': 'raw_document_content'
                }
            ]
        }
    ]
}

In [ ]:
# 测试数据
mock_data = {
    'key_name': 'raw_document',
    'content': raw_document}
print(type(mock_data))
print(mock_data)

In [ ]:
# 和cache里的save_agent_state不同， 前者是一个AgentState对象， agent进行存储时，会存储成AgentStateHistory对象 
# 这里是 tiptap Json对象（没有pydantic约束， 用了dict表示）
await cache.save_document('raw_document', raw_document)


In [ ]:
# 检查django storage 数据是否还在？  理论上应该在
doc_data = await storage.get_from_django(params={'fields': 'raw_document'})
print(doc_data)
print(type(doc_data))

In [ ]:
# 缓存数据是否还在？ 是什么格式
from app.core.redis_helper import RedisClient
key = cache.get_cache_keys().get('raw_document')
doc_data = await RedisClient.get(key)
print(doc_data)
print(type(doc_data))

In [ ]:
# 用get_document工具将直接取得文档内容，不带key_name
doc_data = await cache.get_document('raw_document')
print(doc_data)
print(type(doc_data))

In [ ]:
# 尝试删除缓存
key = cache.get_cache_keys().get('raw_document')
from app.core.redis_helper import RedisClient
await RedisClient.delete(key)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
doc = await cache.get_document('raw_document')
print(doc)

In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
doc = await cache.get_document('raw_document')
print(doc)


#### 3. 测试SSE_MESSAGE的方法

In [8]:
# mock data
# MOCK AgentSSEMessage数据
agent_message_1 = AgentMessage(
    agent_id="123",
    message_id="123",
    message="Hello, world!",
    created_at=datetime.now(),
    updated_at=datetime.now()
)

agent_message_2 = AgentMessage(  
    agent_id="123",
    message_id="123",
    message="Hello, agent!",
    created_at=datetime.now(),
    updated_at=datetime.now()
)

In [ ]:
# 存储AgentSSEMessage数据
await cache.save_agent_message(agent_message_1)

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_message, agent_message_history =  await cache.get_agent_message()
print(agent_message)
print(agent_message_history)

In [ ]:
# 删除缓存
key = cache.get_cache_keys().get('agent_message_history')
from app.core.redis_helper import RedisClient
await RedisClient.delete(key)


In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
# 检查django storage 数据是否还在？  理论上应该在
data = await storage.get_from_django(params={'fields': 'agent_message_history'})
print(data)
print(type(data))

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_message, agent_message_history =  await cache.get_agent_message()
print(agent_message)
print(agent_message_history)

In [ ]:
# 缓存数据是否还在？ 理论上应该不在
data = await RedisClient.get(key)
print(data)

In [ ]:
# 使用 cache的clean up来删除
await cache.clean_up(['agent_message_history'])

In [ ]:
# 状态和状态历史 获取， 验证是否存储成功
agent_message, agent_message_history =  await cache.get_agent_message()
print(agent_message)
print(agent_message_history)

In [ ]:
# 理论上，缓存和storage 数据都不在了
data1 = await RedisClient.get(key)
print(data1)
data2 = await storage.get_from_django(params={'fields': 'agent_message_history'})
print(data2)
print(type(data2))